In [ ]:
# coding: utf-8
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()
import os
import shioaji as sj

api = sj.Shioaji()
api.login(
    api_key=os.getenv('API_KEY1'),
    secret_key=os.getenv('SECRET_KEY1'),
    fetch_contract=False,
)
api.fetch_contracts(contract_download=True)


Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [14]:
api.usage()

UsageStatus(connections=2, bytes=377699446, limit_bytes=524288000, remaining_bytes=146588554)

### 獲取資料

In [7]:
# https://ithelp.ithome.com.tw/articles/10280898 教學參考
def get_shioaji_index_data(begin, end, id, type='1k'):
    # 將 begin 和 end 轉換為所需的格式 2024_0708
    begin_month = begin[:7].replace('-', '')  # 取得2024-07部分，變成202407
    end_month = end[:7].replace('-', '')      # 取得2024-08部分，變成202408
    folder_name = f"{begin[:4]}_{begin_month[4:6]}{end_month[4:6]}"  # 2024_0708
    
    # 定義路徑
    folder_path = f"./index_data/shioaji/{folder_name}/"
    
    # 如果資料夾不存在，則創建
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    # 定義CSV檔案的名稱
    file_name = f"{id}"
    file_path = os.path.join(folder_path, file_name)
    
    #如果要查看合約代號可以直接print(api.Contracts), 查看特定的股票可以print(api.Contracts.Stocks.TSE['2330'])
    # 創建一個空的 DataFrame 來存儲所有數據
    all_ticks = pd.DataFrame()

    if type == '1k':
        print('Remain usage API usage:', api.usage())
            
        # 調用 API 獲取當天的數據
        kbars = api.kbars(
            contract=api.Contracts.Stocks[id],
            start=begin,
            end=end,
        )
            
        df = pd.DataFrame({**kbars})
        all_ticks = pd.concat([all_ticks, df], ignore_index=True)
        
        print(f"Processed data from {begin} to {end}")
        
        all_ticks['ts'] = pd.to_datetime(all_ticks['ts'])

        all_ticks.to_csv(f"{file_path}k.csv", index=False)
        print(f'Data saved to {file_path}')
        
    elif type == 'tick':
        # 將 begin 和 end 轉換為 datetime 對象
        start_date = datetime.strptime(begin, "%Y-%m-%d")
        end_date = datetime.strptime(end, "%Y-%m-%d")
    
        # 循環日期範圍
        current_date = start_date
        while current_date <= end_date:
            date_str = current_date.strftime("%Y-%m-%d")
            print(f"Fetching data for {date_str}..., Remain usage API usage: {api.usage()}")
            
            try:
                # 調用 API 獲取當日數據
                ticks = api.ticks(
                    contract=api.Contracts.Stocks[id],
                    date=date_str
                )
                df = pd.DataFrame({**ticks})
            
                if not df.empty:
                    # 合併當天數據到總表
                    all_ticks = pd.concat([all_ticks, df], ignore_index=True)
                    print(f"Data for {date_str} processed.")
                else:
                    print(f"No data available for {date_str}.")
        
            except Exception as e:
                print(f"Error fetching data for {date_str}: {e}")
        
            # 日期增加一天
            current_date += timedelta(days=1)   
        
        all_ticks['ts'] = pd.to_datetime(all_ticks['ts'])
        all_ticks.to_csv(f"{file_path}.csv", index=False)
        print(f"Data saved to {file_path}")
        
    elif type == 'future-k':
        # 循環日期範圍
        print(f"Fetching data for {begin}, {end}..., Remain usage API usage: {api.usage()}")
        
        try:
            # 調用 API 獲取當日數據
            ticks = api.kbars(
                contract=api.Contracts.Futures[id],
                start=begin,
                end=end
            )
            df = pd.DataFrame({**ticks})
            
            if not df.empty:
                # 合併當天數據到總表
                all_ticks = pd.concat([all_ticks, df], ignore_index=True)
                print(f"Data for {begin}, {end} processed.")
            else:
                print(f"No data available for {begin}, {end}.")
        
        except Exception as e:
            print(f"Error fetching data for {begin}, {end}: {e}")
        
        all_ticks['ts'] = pd.to_datetime(all_ticks['ts'])
        all_ticks.to_csv(f"{file_path}k.csv", index=False)
        print(f"Data saved to {file_path}")
   
    elif type == 'future-t':
        # 將 begin 和 end 轉換為 datetime 對象
        start_date = datetime.strptime(begin, "%Y-%m-%d")
        end_date = datetime.strptime(end, "%Y-%m-%d")
    
        # 循環日期範圍
        current_date = start_date
        while current_date <= end_date:
            date_str = current_date.strftime("%Y-%m-%d")
            print(f"Fetching data for {date_str}..., Remain usage API usage: {api.usage()}")
            
            try:
                # 調用 API 獲取當日數據
                ticks = api.ticks(
                    contract=api.Contracts.Futures[id],
                    date=date_str
                )
                df = pd.DataFrame({**ticks})
            
                if not df.empty:
                    # 合併當天數據到總表
                    all_ticks = pd.concat([all_ticks, df], ignore_index=True)
                    print(f"Data for {date_str} processed.")
                else:
                    print(f"No data available for {date_str}.")
        
            except Exception as e:
                print(f"Error fetching data for {date_str}: {e}")
        
            # 日期增加一天
            current_date += timedelta(days=1)
        
        all_ticks['ts'] = pd.to_datetime(all_ticks['ts'])
        all_ticks.to_csv(f"{file_path}.csv", index=False)
        print(f"Data saved to {file_path}")

    elif type == 'index-t':
        # 設置開始日期和結束日期
        start_date, end_date = datetime.strptime(begin, "%Y-%m-%d"), datetime.strptime(end, "%Y-%m-%d")
        
        # 遍歷日期範圍，按天調用 API
        current_date = start_date
        
        while current_date <= end_date:
            print('Remain usage API usage:', api.usage())

            # 格式化日期為字符串
            date_str = current_date.strftime("%Y-%m-%d")

            # 調用 API 獲取當天的數據 TSE001(大盤), TSE099(台灣50)
            ticks = api.ticks(
                contract=api.Contracts.Indexs.TSE.TSE001,
                date=date_str
            )

            df = pd.DataFrame({**ticks})
            all_ticks = pd.concat([all_ticks, df], ignore_index=True)

            print(f"Processed data for {date_str}")

            current_date += timedelta(days=1)

        all_ticks = all_ticks[['ts', 'close']]
        all_ticks['ts'] = pd.to_datetime(all_ticks['ts'])

        all_ticks.to_csv(f"{file_path}.csv", index=False)
        print(f'Data saved to {file_path}')

    elif type == 'index-k':
        # 循環日期範圍
        print(f"Fetching data for {begin}, {end}..., Remain usage API usage: {api.usage()}")
        
        try:
            # 調用 API 獲取當日數據
            ticks = api.kbars(
                contract=api.Contracts.Indexs.TSE.TSE001,
                start=begin,
                end=end
            )
            df = pd.DataFrame({**ticks})
            
            if not df.empty:
                # 合併當天數據到總表
                all_ticks = pd.concat([all_ticks, df], ignore_index=True)
                print(f"Data for {begin}, {end} processed.")
            else:
                print(f"No data available for {begin}, {end}.")
        
        except Exception as e:
            print(f"Error fetching data for {begin}, {end}: {e}")
        
        all_ticks['ts'] = pd.to_datetime(all_ticks['ts'])
        all_ticks.to_csv(f"{file_path}k.csv", index=False)
        print(f"Data saved to {file_path}")

    return


In [8]:
# get_shioaji_index_data('2025-04-01', '2025-06-06', '2881', type='tick')
# get_shioaji_index_data('2025-04-01', '2025-06-06', '2881', type='1k')
# get_shioaji_index_data('2025-04-01', '2025-06-06', '2882', type='tick')
# get_shioaji_index_data('2025-04-01', '2025-06-06', '2882', type='1k')

# get_shioaji_index_data('2025-04-01', '2025-05-30', 'TSE001', type='index-t')
# get_shioaji_index_data('2025-04-01', '2025-05-30', 'TSE001', type='index-k')

# get_shioaji_index_data('2025-04-01', '2025-06-05', 'TMFR1', type='future-t')
# get_shioaji_index_data('2025-04-01', '2025-06-05', 'TMFR1', type='future-k')
get_shioaji_index_data('2025-04-01', '2025-06-05', 'MXFR1', type='future-t')
get_shioaji_index_data('2025-04-01', '2025-06-05', 'MXFR1', type='future-k')

Fetching data for 2025-04-01..., Remain usage API usage: connections=2 bytes=28856726 limit_bytes=524288000 remaining_bytes=495431274
Data for 2025-04-01 processed.
Fetching data for 2025-04-02..., Remain usage API usage: connections=2 bytes=28858505 limit_bytes=524288000 remaining_bytes=495429495
Data for 2025-04-02 processed.
Fetching data for 2025-04-03..., Remain usage API usage: connections=1 bytes=36116729 limit_bytes=524288000 remaining_bytes=488171271
Data for 2025-04-03 processed.
Fetching data for 2025-04-04..., Remain usage API usage: connections=1 bytes=43030317 limit_bytes=524288000 remaining_bytes=481257683
No data available for 2025-04-04.
Fetching data for 2025-04-05..., Remain usage API usage: connections=1 bytes=43030317 limit_bytes=524288000 remaining_bytes=481257683
Data for 2025-04-05 processed.
Fetching data for 2025-04-06..., Remain usage API usage: connections=1 bytes=47918450 limit_bytes=524288000 remaining_bytes=476369550
Data for 2025-04-06 processed.
Fetchin

In [12]:
api.logout()

True

### 查看合約與指數

In [2]:
# 遍歷所有期貨代碼
for future_code in dir(api.Contracts.Futures):
    # 排除內建屬性和方法，只保留商品代碼
    if not future_code.startswith('__'):
        try:
            # 構建近月合約的完整代碼 (例如 FZ1.FZ1R1)
            contract = getattr(api.Contracts.Futures, future_code)
            near_month_code = f"{future_code}R1"
            future_detail = getattr(contract, near_month_code)
            
            # 打印詳細資訊
            print(f"商品代碼: {future_detail.code}")
            print(f"符號: {future_detail.symbol}")
            print(f"名稱: {future_detail.name}")
            print(f"類別: {future_detail.category}")
            print(f"交割月份: {future_detail.delivery_month}")
            print(f"交割日期: {future_detail.delivery_date}")
            print(f"標的種類: {future_detail.underlying_kind}")
            print(f"標的代碼: {future_detail.underlying_code}")
            print(f"單位: {future_detail.unit}")
            print(f"漲停價: {future_detail.limit_up}")
            print(f"跌停價: {future_detail.limit_down}")
            print(f"參考價: {future_detail.reference}")
            print(f"更新日期: {future_detail.update_date}")
            print(f"目標代碼: {future_detail.target_code}")
            print("-" * 50)  # 分隔線
            
        except AttributeError:
            # 如果某個代碼沒有 R1 合約，跳過並繼續
            print(f"{future_code} 沒有近月合約數據")
            continue
        except Exception as e:
            # 處理其他可能的錯誤
            print(f"處理 {future_code} 時發生錯誤: {str(e)}")
            continue

商品代碼: BRFR1
符號: BRFR1
名稱: 布蘭特原油期貨近月
類別: BRF
交割月份: 202504
交割日期: 2025/04/02
標的種類: C
標的代碼: 
單位: 1
漲停價: 2431.0
跌停價: 2200.0
參考價: 2315.5
更新日期: 2025/03/20
目標代碼: BRFE5
--------------------------------------------------
商品代碼: BTFR1
符號: BTFR1
名稱: 臺灣生技期貨近月
類別: BTF
交割月份: 202504
交割日期: 2025/04/16
標的種類: I
標的代碼: 
單位: 1
漲停價: 4259.0
跌停價: 3485.0
參考價: 3872.0
更新日期: 2025/03/20
目標代碼: BTFD5
--------------------------------------------------
商品代碼: CAFR1
符號: CAFR1
名稱: 南亞期貨近月
類別: CAF
交割月份: 202504
交割日期: 2025/04/16
標的種類: S
標的代碼: 1303
單位: 1
漲停價: 37.6
跌停價: 30.8
參考價: 34.2
更新日期: 2025/03/20
目標代碼: CAFD5
--------------------------------------------------
商品代碼: CBFR1
符號: CBFR1
名稱: 中鋼期貨近月
類別: CBF
交割月份: 202504
交割日期: 2025/04/16
標的種類: S
標的代碼: 2002
單位: 1
漲停價: 26.45
跌停價: 21.65
參考價: 24.05
更新日期: 2025/03/20
目標代碼: CBFD5
--------------------------------------------------
商品代碼: CCFR1
符號: CCFR1
名稱: 聯電期貨近月
類別: CCF
交割月份: 202504
交割日期: 2025/04/16
標的種類: S
標的代碼: 2303
單位: 1
漲停價: 49.0
跌停價: 40.1
參考價: 44.55
更新日期: 2025/03/20
目標代碼: CCFD5
--------

In [4]:
print(api.Contracts.Indexs.TSE)

for i in range(1, 100):
    contract_code = f"{i:03}"  # 格式化為三位數字
    contract = api.Contracts.Indexs.TSE.get(contract_code)  # 使用 get() 方法避免 KeyError
    if contract and hasattr(contract, 'name'):  # 確保 contract 存在且有 name 屬性
        print(contract.name)
    else:
        print(f"Contract TSE{contract_code} not found.")

TSE(TSE001, TSE002, TSE003, TSE004, TSE005, TSE006, TSE007, TSE008, TSE009, TSE010, TSE011, TSE012, TSE014, TSE015, TSE016, TSE017, TSE018, TSE019, TSE020, TSE021, TSE022, TSE023, TSE024, TSE025, TSE026, TSE027, TSE028, TSE029, TSE030, TSE031, TSE032, TSE033, TSE034, TSE035, TSE036, TSE037, TSE038, TSE039, TSE040, TSE041, TSE042, TSE043, TSE046, TSE047, TSE048, TSE049, TSE050, TSE051, TSE052, TSE053, TSE054, TSE055, TSE056, TSE057, TSE058, TSE059, TSE060, TSE061, TSE062, TSE064, TSE066, TSE067, TSE068, TSE069, TSE070, TSE071, TSE072, TSE073, TSE074, TSE076, TSE077, TSE079, TSE082, TSE083, TSE084, TSE085, TSE087, TSE088, TSE089, TSE090, TSE091, TSE092, TSE093, TSE094, TSE095, TSE096, TSE097, TSE098, TSE099)
加權指數
不含金融指數
不含電子指數
化學工業
生技醫療業
水泥窯製
食品類股
塑膠化工
紡織纖維
機電類
造紙類
建材營造類指數
Contract TSE013 not found.
金融保險類指數
水泥工業
食品工業
塑膠工業
紡織纖維
電機機械
電器電纜
化學生技醫療
玻璃陶瓷
造紙工業
鋼鐵工業
橡膠工業
汽車工業
電子工業
建材營造
航運業
觀光事業
金融保險
貿易百貨
其他
未含金電股發行量加權股價指數
油電燃氣業
半導體業
電腦及週邊設備業
光電業
通信網路業
電子零組件業
電子通路業
資訊服務業
其他電子業
Contract TSE044 not